In [0]:
/FileStore/tables/menu_csv.txt
/FileStore/tables/sales_csv.txt

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4225644477721937>:1
----> 1 FileStore/tables/menu_csv.txt()
      2 FileStore/tables/sales_csv.txt()

NameError: name 'FileStore' is not defined

In [0]:
from pyspark.sql.types import StructType, IntegerType, DoubleType, DateType, StringType, StructField

In [0]:
schema = StructType([StructField("Product_ID", IntegerType(), True),
                    StructField("Customer_ID", StringType(), True),
                    StructField("Order_Date", DateType(), True),
                    StructField("Location", StringType(), True),
                    StructField("Source_Order", StringType(), True)
                    ]
)

Sales_df = spark.read.format('csv').option('inferschema',"True").schema(schema).load('/FileStore/tables/sales_csv.txt')
display(Sales_df)

In [0]:
from pyspark.sql.functions import month, year, quarter



Sales_df = Sales_df.withColumn("Order_Year", year(Sales_df.Order_Date)) \
           .withColumn("Order_month", month(Sales_df.Order_Date)) \
           .withColumn("Order_Quarter", quarter(Sales_df.Order_Date))


In [0]:
display(Sales_df)

In [0]:
schema = StructType([StructField("Product_ID", IntegerType(), True),
                    StructField("Product_Name", StringType(), True),
                    StructField("Price", StringType(), True),
                    ]
)

Menu_df = spark.read.format('csv').option('inferschema',"True").schema(schema).load('/FileStore/tables/menu_csv.txt')
display(Menu_df)

In [0]:
Total_amt_Cust = Sales_df.join(Menu_df,"Product_Id",how = "inner").groupBy("Customer_ID").agg({'Price':'sum'}).orderBy('Customer_Id')

display(Total_amt_Cust)

In [0]:
Total_amt_food = Sales_df.join(Menu_df,on= 'Product_Id', how = 'inner').groupBy('Product_Name').agg({'Price':'sum'}).orderBy('Product_Name')

display(Total_amt_food)

Product_Name,sum(Price)
Biryani,480.0
Chowmin,3600.0
Dosa,1320.0
PIZZA,2100.0
Pasta,1080.0
sandwich,5760.0


Databricks visualization. Run in Databricks to view.

In [0]:

Total_Sales_Month = Sales_df.join(Menu_df, on = 'Product_Id', how = 'inner').groupBy('Order_month').agg({'Price':'sum'}).orderBy('Order_month')

display(Total_Sales_Month)

In [0]:
Total_Sales_Yearly = Sales_df.join(Menu_df, on = 'Product_Id', how = 'inner').groupBy('Order_Year').agg({'Price':'sum'}).orderBy('Order_Year')

display(Total_Sales_Yearly)

In [0]:
Total_Sales_Quaterly = Sales_df.join(Menu_df, on = 'Product_Id', how = 'inner').groupBy('Order_Quarter').agg({'Price':'sum'}).orderBy('Order_Quarter')

display(Total_Sales_Quaterly)

In [0]:
from pyspark.sql.functions import count

Total_order = Sales_df.join(Menu_df, on = 'Product_Id', how= 'inner').groupBy(['Product_Id','Product_Name']).agg(count('Product_Id').alias('Total_Orders')).orderBy('Total_Orders', ascending = 0)

display(Total_order)


Product_Id,Product_Name,Total_Orders
3,sandwich,48
2,Chowmin,24
1,PIZZA,21
4,Dosa,12
5,Biryani,6
6,Pasta,6


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import count

Total_5_order = Sales_df.join(Menu_df, on = 'Product_Id', how= 'inner').groupBy(['Product_Id','Product_Name']).agg(count('Product_Id').alias('Total_Orders')).orderBy('Total_Orders', ascending = 0).limit(5)

display(Total_5_order)

In [0]:
from pyspark.sql.functions import count

Top_ordered = Sales_df.join(Menu_df, on = 'Product_Id', how= 'inner').groupBy(['Product_Id','Product_Name']).agg(count('Product_Id').alias('Total_Orders')).orderBy('Total_Orders', ascending = 0).limit(1)

display(Top_ordered)

In [0]:
from pyspark.sql.functions import countDistinct

Freq_df = Sales_df.filter((Sales_df.Source_Order == 'Restaurant')).groupBy('Customer_Id').agg(countDistinct('Order_Date').alias('Frequency')).orderBy('Customer_Id')

display(Freq_df)

In [0]:
Total_Sales_Country = Sales_df.join(Menu_df,on= 'Product_Id', how = 'inner').groupBy('Location').agg({'Price':'sum'}).orderBy('Location')

display(Total_Sales_Country)

In [0]:
Total_Sales_Source = Sales_df.join(Menu_df,on= 'Product_Id', how = 'inner').groupBy('Source_Order').agg({'Price':'sum'}).orderBy('Source_Order')

display(Total_Sales_Source)